# WeatherAPI


---

# Generate Random Geographic Coordinates and a List of Cities

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

/Users/jnorth/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 592


## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [3]:
import json

# Set the API base URL
url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat_lng[0]}&lon={lat_lng[1]}&appid={weather_api_key}"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}"
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        response = requests.get(city_url)

        city_data = response.json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_data["coord"]["lat"]
        city_lng = city_data["coord"]["lon"]
        city_max_temp = city_data["main"]["temp_max"]
        city_humidity = city_data["main"]["humidity"]
        city_clouds = city_data["clouds"]["all"]
        city_wind = city_data["wind"]["speed"]
        city_country = city_data["sys"]["country"]
        city_date = city_data["dt"]

        print(city_country)

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | saldanha
286.04
City not found. Skipping...
Processing Record 2 of Set 1 | karkira
298.4
City not found. Skipping...
Processing Record 3 of Set 1 | codajas
295.93
City not found. Skipping...
Processing Record 4 of Set 1 | college
300.12
City not found. Skipping...
Processing Record 5 of Set 1 | talcahuano
279.97
City not found. Skipping...
Processing Record 6 of Set 1 | ushuaia
273.96
City not found. Skipping...
Processing Record 7 of Set 1 | port-aux-francais
276.93
City not found. Skipping...
Processing Record 8 of Set 1 | adamstown
294.86
City not found. Skipping...
Processing Record 9 of Set 1 | bethel
292.37
City not found. Skipping...
Processing Record 10 of Set 1 | blackwater
295.2
City not found. Skipping...
Processing Record 11 of Set 1 | olonkinbyen
278.34
City not found. Skipping...
Processing Record 12 of Set 1 | saint-philippe
297.92
City not found. Skipping...
Processing Record 13 o

283.55
City not found. Skipping...
Processing Record 17 of Set 1 | rio rancho
302.51
City not found. Skipping...
Processing Record 18 of Set 1 | sungai penuh


In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df